In [ ]:
%matplotlib inline

In [ ]:
import pickle
import matplotlib
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns; sns.set_context('notebook')
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
Characterisation_Set = pickle.load(open("Battery_Data/new_battery_cycles/Characterisation_Set_Complete.p", 'rb'))

def scale(X, torch=False):
    """

    :param X:
    :return: Normalised array like X, mean, std
    """
    if torch:
        raise Exception("Not implement yet")
    else:
        X_min = X.min()
        X_max = X.max()
        
    return (X - X_min)/(X_max - X_min), X_min, X_max   

In [ ]:
SoC, SoC_min, SoC_max = scale(Characterisation_Set["SoC"].T)
Current, Current_min, Current_max = scale(Characterisation_Set["Current"].T)
# Voltage, Voltage_min, Voltage_max = scale(Characterisation_Set["Voltage"].T)
Voltage = Characterisation_Set["Voltage"].T
Characterisation_Set["preprocessing"] = {
    "SoC": (SoC_max, SoC_min),
    "Current": (Current_max, Current_min)
}

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
plt.figure("Characterisation_Set")
plt.subplot(311)
plt.plot(SoC, '-g')
plt.subplot(312)
plt.plot(Current, '-b')
plt.subplot(313)
plt.plot(Voltage, '-r')
plt.show()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
plt.plot(Current*(Current_max - Current_min) + Current_min, '-b')
plt.show()

In [ ]:
class Prior_RNN(nn.Module):
    def __init__(self, p=0.25):
        super(Battery_RNN, self).__init__()
        self.p = p
        self.Z_hl1 = nn.Linear(2, 1024)
        self.Z_hl2 = nn.Linear(1024, 512)
        self.Z_p = nn.Linear(512, 1)
        
        self.Voc_hl1 = nn.Linear(1, 1024)
        self.Voc_hl2 = nn.Linear(1024, 512)
        self.VoC = nn.Linear(512, 1)
                
    def forward(self, soc_init, current):
        
        voltage = torch.empty(current.shape, dtype=torch.float)
        soc_hist = torch.empty(current.shape, dtype=torch.float)
        soc = torch.Tensor([[soc_init]])
        soc = soc.to(device, torch.float)
        for t in range(current.shape[1]):
            I = torch.Tensor([[current[0, t]]])
            I = I.to(device, torch.float)
            # Estimate VoC
            VoC = torch.sigmoid(self.Voc_hl1(soc))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = torch.sigmoid(self.Voc_hl2(VoC))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = self.VoC(VoC)

            # Estimate Z_p
            combined = torch.cat((soc, I), 1)
            Z = torch.sigmoid(self.Z_hl1(combined))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = torch.sigmoid(self.Z_hl2(Z))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = self.Z_p(Z)

            # Estimate V
            scaled_I = I*(Current_max - Current_min) + Current_min
            V = VoC - scaled_I*Z
            voltage[0, t] = V

            # Predict SoC
            soc = soc - scaled_I*V/Characterisation_Set['E_crit']
            soc_hist[0, t] = soc
            
            max_test = soc[:, 0] > 1.0
            soc[max_test, 0] = 1.0
            min_test = soc[:, 0] < 0.0
            soc[min_test, 0] = 0.0000000001          

        
        return voltage, soc_hist

In [ ]:
class Battery_RNN(nn.Module):
    def __init__(self, p=0.25):
        super(Battery_RNN, self).__init__()
        self.p = p
        self.Z_hl1 = nn.Linear(2, 1024)
        self.Z_hl2 = nn.Linear(1024, 512)
        self.Z_p = nn.Linear(512, 1)
        
        self.Voc_hl1 = nn.Linear(1, 1024)
        self.Voc_hl2 = nn.Linear(1024, 512)
        self.VoC = nn.Linear(512, 1)
                
    def forward(self, soc_init, current):
        
        voltage = torch.empty(current.shape, dtype=torch.float)
        soc_hist = torch.empty(current.shape, dtype=torch.float)
        soc = torch.Tensor([[soc_init]])
        soc = soc.to(device, torch.float)
        for t in range(current.shape[1]):
            I = torch.Tensor([[current[0, t]]])
            I = I.to(device, torch.float)
            # Estimate VoC
            VoC = torch.sigmoid(self.Voc_hl1(soc))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = torch.sigmoid(self.Voc_hl2(VoC))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = self.VoC(VoC)

            # Estimate Z_p
            combined = torch.cat((soc, I), 1)
            Z = torch.sigmoid(self.Z_hl1(combined))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = torch.sigmoid(self.Z_hl2(Z))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = self.Z_p(Z)

            # Estimate V
            scaled_I = I*(Current_max - Current_min) + Current_min
            V = VoC - scaled_I*Z
            voltage[0, t] = V

            # Predict SoC
            soc = soc - scaled_I*V/Characterisation_Set['E_crit']
            soc_hist[0, t] = soc
            
            max_test = soc[:, 0] > 1.0
            soc[max_test, 0] = 1.0
            min_test = soc[:, 0] < 0.0
            soc[min_test, 0] = 0.0000000001          

        
        return voltage, soc_hist

In [ ]:
battery = Battery_RNN()

In [ ]:
# init
for W in battery.parameters():
    nn.init.normal_(W)

In [ ]:
# Loss and optimizer
criterion = nn.MSELoss()# Mean Squared Loss
# The weight_decay option implements L2 regularisation
optimizer = optim.Adam(battery.parameters()) 

In [ ]:
# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
battery.to(device)

In [ ]:
import time
import math

epochs = 2500
running_loss = 0.0
loss_min = 1e10
loss_hist = []
    
battery.eval()
start = time.time()

def timeSince(since):
    now = time.time()
    interval = now - since
    m = math.floor(interval / 60)
    s = interval - m * 60
    return '%dm %ds' % (m, s), now, interval

for epoch in range(epochs):
    count = -1
    avg_loss = 0
    loss_set_hist = []
    
    for set_dict in Characterisation_Set['Sets']:
        count += 1
        V = torch.from_numpy(set_dict['Voltage']).float()
        optimizer.zero_grad()
        V_est, SoC = battery(set_dict['SoC'][0, 0], set_dict['Current'])
        loss = criterion(V_est, V)
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()
        loss_set_hist.append(loss.item())
    loss_hist.append(loss_set_hist)
    avg_loss /= (count+1)
    if epoch % 5 == 0:
        now_string, now, interval = timeSince(start)
        remaining_epochs = epochs-(epoch+1)
        remaining_time = interval*remaining_epochs/(epoch + 1)
        h_f = remaining_time / 60.0 / 60.0
        h = math.floor(h_f)
        m_f = (h_f - h)*60.0
        m = math.floor(m_f)
        s = (m_f - m)*60.0
        remaining_string = '%dh %dm %ds' % (h, m, s)
        print("epoch {}, time since start: {}, estimated remaining time: {}".format(epoch, now_string, remaining_string))
    if avg_loss < loss_min:
        print("New average minimum: ", avg_loss)
        torch.save(battery.state_dict(), "./Battery_Data/new_battery_cycles/Battery_RNN_v3_no_drop_bounded.mdl")
        loss_min = avg_loss
            

In [ ]:
battery = Battery_RNN()
battery.load_state_dict(torch.load("./Battery_Data/new_battery_cycles/Battery_RNN_v3_no_drop_bounded.mdl"))
battery.to(device)
battery.eval()

with torch.no_grad():
    for i, set_dict in enumerate(Characterisation_Set['Sets']):
        
        V = torch.from_numpy(set_dict['Voltage']).float()
        V_est, SoC = battery(set_dict['SoC'][0, 0], set_dict['Current'])
        plt.figure(i)
        plt.subplot(2,1,1)
        plt.plot(set_dict['SoC'].T, 'k')
        plt.plot(SoC.to("cpu").numpy().T, 'g')
        plt.subplot(2,1,2)
        plt.plot(V.to("cpu").numpy().T, 'k')
        plt.plot(V_est.to("cpu").numpy().T, 'b')
        

In [ ]:
class Explore_Battery_RNN(nn.Module):
    def __init__(self, p=0.25):
        super(Battery_RNN, self).__init__()
        self.p = p
        self.Z_hl1 = nn.Linear(2, 1024)
        self.Z_hl2 = nn.Linear(1024, 512)
        self.Z_p = nn.Linear(512, 1)
        
        self.Voc_hl1 = nn.Linear(1, 1024)
        self.Voc_hl2 = nn.Linear(1024, 512)
        self.VoC = nn.Linear(512, 1)
        
    def VoC_est(self, soc):
        VoC = torch.sigmoid(self.Voc_hl1(soc))
        VoC = F.dropout(VoC, training=self.training, p=self.p)
        VoC = torch.sigmoid(self.Voc_hl2(VoC))
        VoC = F.dropout(VoC, training=self.training, p=self.p)
        VoC = self.VoC(VoC)
        return VoC
    
    def Z_est(self, soc, I):
        combined = torch.cat((soc, I), 1)
        Z = torch.sigmoid(self.Z_hl1(combined))
        Z = F.dropout(Z, training=self.training, p=self.p)
        Z = torch.sigmoid(self.Z_hl2(Z))
        Z = F.dropout(Z, training=self.training, p=self.p)
        Z = self.Z_p(Z)
        return Z

    def forward(self, soc_init, current):
        
        voltage = torch.empty(current.shape, dtype=torch.float)
        soc_hist = torch.empty(current.shape, dtype=torch.float)
        soc = torch.Tensor([[soc_init]])
        soc = soc.to(device, torch.float)
        for t in range(current.shape[1]):
            I = torch.Tensor([[current[0, t]]])
            I = I.to(device, torch.float)
            # Estimate VoC
            VoC = torch.sigmoid(self.Voc_hl1(soc))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = torch.sigmoid(self.Voc_hl2(VoC))
            VoC = F.dropout(VoC, training=self.training, p=self.p)
            VoC = self.VoC(VoC)

            # Estimate Z_p
            combined = torch.cat((soc, I), 1)
            Z = torch.sigmoid(self.Z_hl1(combined))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = torch.sigmoid(self.Z_hl2(Z))
            Z = F.dropout(Z, training=self.training, p=self.p)
            Z = self.Z_p(Z)

            # Estimate V
            V = VoC - I*Z
            voltage[0, t] = V

            # Predict SoC
            soc = soc - I*V/Characterisation_Set['E_crit']
            soc_hist[0, t] = soc
        
        return voltage, soc_hist